In [61]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [62]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [63]:
#Query data from order_logs_snapshot

q = """
select yyyymmdd, epoch, data_orderId, data_propagationBatchId, data_riderId, eventType, hh, updated_epoch from raw.kafka_dispatch_propagation_immutable
where data_serviceDetailId = '5fc8c40bf4f0a70007d7cfbc'
and yyyymmdd in ('20230620')
and hh in ('17', '18', '19', '20', '21')
and eventType = 'rider_acknowledged'
"""


#Load data into pandas table
df_propagation_copy = pd.read_sql(q, connection)

In [64]:
df_propagation = df_propagation_copy.copy()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
0,20230620,1687277920300,6491d160db4fa57679b2c53f,69d11ae1-102f-4b16-bc50-a52d3279e938,5a8db9a59e7fc44257057ff7,rider_acknowledged,21,1687289939651
1,20230620,1687277920300,6491d160db4fa57679b2c53f,10e33407-3bbb-42a4-a3c2-a8aa5788db0d,62555d2fb4846f7ca08b98e9,rider_acknowledged,21,1687289439460
2,20230620,1687277920300,6491d160db4fa57679b2c53f,10e33407-3bbb-42a4-a3c2-a8aa5788db0d,644f0f3dce2928788c2d9d88,rider_acknowledged,21,1687289438688
3,20230620,1687277920300,6491d160db4fa57679b2c53f,10e33407-3bbb-42a4-a3c2-a8aa5788db0d,6337e3faabf2101bf52a7f4c,rider_acknowledged,21,1687289440929
4,20230620,1687277920300,6491d160db4fa57679b2c53f,10e33407-3bbb-42a4-a3c2-a8aa5788db0d,60cd8525054689f1b6db540c,rider_acknowledged,21,1687289438983


In [65]:
df_propagation = df_propagation.sort_values(by = ['data_orderId', 'updated_epoch', 'data_propagationBatchId'])
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
114456,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,62b428a17d79674d27f10930,rider_acknowledged,17,1687260601815
73736,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,641a7d389f4a35e82baaa04b,rider_acknowledged,17,1687260602457
170428,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,5fedb930bd3aea2f3338de42,rider_acknowledged,17,1687260603460
160177,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,63bfa74743fda772df186fc9,rider_acknowledged,17,1687260604392
170784,20230620,1687260601523,64918db9fc04cd243acab67c,b42eb3d4-3740-4e33-9fdb-62eef223e71a,63673c22237e84e9d0a532a1,rider_acknowledged,17,1687260603190


In [66]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
114456,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,62b428a17d79674d27f10930,rider_acknowledged,17,1687260601815,1
73736,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,641a7d389f4a35e82baaa04b,rider_acknowledged,17,1687260602457,1
170428,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,5fedb930bd3aea2f3338de42,rider_acknowledged,17,1687260603460,1
160177,20230620,1687260600315,64918db8b2f49b01869c2922,5131b8b1-c983-4ace-9053-a96c9660707c,63bfa74743fda772df186fc9,rider_acknowledged,17,1687260604392,1
170784,20230620,1687260601523,64918db9fc04cd243acab67c,b42eb3d4-3740-4e33-9fdb-62eef223e71a,63673c22237e84e9d0a532a1,rider_acknowledged,17,1687260603190,1


In [67]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8, estimated_accept_to_pickup_distance
from orders.order_logs_immutable
where yyyymmdd = '20230620'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and city_name = 'Delhi'
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [68]:
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance
0,64919156ce708168802b927c,NaN,883da11515fffff,28.569002,77.212482,Delhi,883da1153bfffff,883da115a1fffff,28.556202,77.253891,28.568962,77.212784,171526,17,1715,Auto,17:00,2,20230620,new,None,1687261548275,rider_busy,61c2f79f17dd6acc5e1afe52,None,883da11539fffff,477.8
1,64918f5ee96d2151db2b378a,NaN,883da10217fffff,28.522977,77.191424,Delhi,883da102ebfffff,883da10287fffff,28.514011,77.234428,28.522774,77.207802,170702,17,1700,Auto,17:00,2,20230620,new,None,1687261129163,rider_busy,611cb5c1890209015a52a4e7,None,883da102ebfffff,1965.0
2,649192cca123ae7aadac84f3,NaN,883da111abfffff,28.495947,77.092732,Delhi,883da111a7fffff,883da11ab1fffff,28.451122,77.087659,28.489159,77.092995,172140,17,1715,Auto,17:00,2,20230620,new,None,1687261982036,rider_busy,614489def2848c83299a14e0,None,883da111a7fffff,1040.7
3,6491901fafae71062c25c42c,NaN,883da1121dfffff,28.618930,77.025059,Delhi,883da11203fffff,883da18dabfffff,28.655682,77.117676,28.610186,77.023376,171015,17,1700,Auto,17:00,2,20230620,new,None,1687261352256,rider_busy,5feca9a7857d1b119621ec32,None,883da11203fffff,1422.5
4,64919423e80c503f0a6d4726,NaN,883da112d9fffff,28.640282,77.082558,Delhi,883da112d1fffff,883da1176dfffff,28.610507,77.114565,28.637908,77.090801,172723,17,1715,Auto,17:00,2,20230620,onTheWay,None,1687262254108,accepted,6385d2484c014e83f4b30394,None,883da112d1fffff,1117.4


In [69]:
df_propagation = df_propagation.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df_propagation_filter = df_propagation[['order_id', 'captain_id', 'counter']]
df_propagation_filter = df_propagation_filter.drop_duplicates()
df_apr = df_apr.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df_accepted = df_apr[df_apr['event_type'] == 'accepted']
df_accepted.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,counter
6,64919423e80c503f0a6d4726,NaN,883da112d9fffff,28.640282,77.082558,Delhi,883da112d1fffff,883da1176dfffff,28.610507,77.114565,28.637908,77.090801,172723,17,1715,Auto,17:00,2,20230620,onTheWay,None,1687262254108,accepted,6385d2484c014e83f4b30394,None,883da112d1fffff,1117.4,1.0
23,64919743fc04cd243acac84f,NaN,883da110c3fffff,28.577063,77.110474,Delhi,883da110ddfffff,883da115c3fffff,28.588634,77.253258,28.587704,77.118866,174043,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687263123431,accepted,600b92e97143235ff5c0b23a,None,883da110ddfffff,1379.8,3.0
25,6491955cc1b4474a4f66950c,NaN,883da11ab5fffff,28.443293,77.080841,Delhi,883da11ab5fffff,883da11abbfffff,28.453547,77.089450,28.442722,77.077400,173236,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687262561819,accepted,5fe2ddce026a4044e4ca9b91,None,883da11ab5fffff,458.7,1.0
26,64919875e205f4706879cc72,NaN,883da11ae3fffff,28.452248,77.050560,Delhi,883da11ae3fffff,883da11a83fffff,28.458678,77.071876,28.453086,77.044795,174549,17,1745,Auto,17:59,2,20230620,onTheWay,None,1687263355036,accepted,634a3b2093d5ea24f7438b0d,None,883da11ae3fffff,807.1,1.0
28,6491975ba123ae7aadac8d9a,NaN,883da1062bfffff,28.569397,77.327637,Delhi,883da1062bfffff,883da10629fffff,28.567341,77.321130,28.568413,77.327118,174107,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687263075627,accepted,5dcae1680856f1461826a50a,None,883da1062bfffff,213.3,1.0


In [70]:
q = """
select * from datasets.hex_adjacency
where city_name = 'delhi'
"""

df_adjacency_copy = pd.read_sql(q, connection)
df_adjacency_copy.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,multicast_config,city_name
0,883da1232dfffff-Saturday-evening_peak,Saturday,evening_peak,883da1232dfffff,"['883da12321fffff', '883da1232dfffff', '883da1...","['883da138dbfffff', '883da13899fffff', '883da1...",2022_reset,17:00,22:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
1,883da13ac5fffff-Sunday-rest_evening,Sunday,rest_evening,883da13ac5fffff,"['883da13acdfffff', '883da13ac7fffff', '883da1...","['883da13aedfffff', '883da13127fffff', '883da1...",2022_reset,22:00,23:59,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
2,883da1232dfffff-Saturday-afternoon,Saturday,afternoon,883da1232dfffff,"['883da12321fffff', '883da1232dfffff', '883da1...","['883da138dbfffff', '883da13899fffff', '883da1...",2022_reset,12:00,17:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
3,883da1a825fffff-Friday-rest_morning,Friday,rest_morning,883da1a825fffff,"['883da1a91bfffff', '883da1a82dfffff', '883da1...","['883da1a821fffff', '883da1a82dfffff', '883da1...",2022_reset,00:00,08:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
4,883da1040dfffff-Friday-evening_peak,Friday,evening_peak,883da1040dfffff,"['883da10409fffff', '883da10405fffff', '883da1...","['883da10409fffff', '883da10407fffff', '883da1...",2022_reset,17:00,22:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi


In [71]:
df_adjacency = df_adjacency_copy.copy()
df_adjacency.head()

,uid,dayname,time_period,source_hex,adjacency,backup_adjacency,type,start_ts,end_ts,city_id,date,propagation_type,propagation_eta_buckets,multicast_max_batch_size,service_detail_id,undesirable_hexes,max_mmo_parallel_order,propagation_score_buckets,on_ride_booking_enabled,pap_enabled,pap_batch_window_in_ms,batch_adjacency,route_preference_eligibility_threshold_score,multicast_config,city_name
0,883da1232dfffff-Saturday-evening_peak,Saturday,evening_peak,883da1232dfffff,"['883da12321fffff', '883da1232dfffff', '883da1...","['883da138dbfffff', '883da13899fffff', '883da1...",2022_reset,17:00,22:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
1,883da13ac5fffff-Sunday-rest_evening,Sunday,rest_evening,883da13ac5fffff,"['883da13acdfffff', '883da13ac7fffff', '883da1...","['883da13aedfffff', '883da13127fffff', '883da1...",2022_reset,22:00,23:59,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
2,883da1232dfffff-Saturday-afternoon,Saturday,afternoon,883da1232dfffff,"['883da12321fffff', '883da1232dfffff', '883da1...","['883da138dbfffff', '883da13899fffff', '883da1...",2022_reset,12:00,17:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
3,883da1a825fffff-Friday-rest_morning,Friday,rest_morning,883da1a825fffff,"['883da1a91bfffff', '883da1a82dfffff', '883da1...","['883da1a821fffff', '883da1a82dfffff', '883da1...",2022_reset,00:00,08:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,5e27db384524755456cdf620,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi
4,883da1040dfffff-Friday-evening_peak,Friday,evening_peak,883da1040dfffff,"['883da10409fffff', '883da10405fffff', '883da1...","['883da10409fffff', '883da10407fffff', '883da1...",2022_reset,17:00,22:00,5bc5ac2312477c2ece769591,20211110,multicast,"[[0,6],[6,12],[12,18]]",4,None,None,3,"[[-1,-0.5],[-0.5,0],[0,0.5],[0.5,1]]",False,False,5,,0.0,,delhi


In [72]:
df_adjacency_filter = df_adjacency[(df_adjacency['time_period'] == 'evening_peak') & (df_adjacency['service_detail_id'] == '5fc8c40bf4f0a70007d7cfbc')]
df_adjacency_filter = df_adjacency_filter[['source_hex', 'multicast_max_batch_size']]
df_adjacency_filter = df_adjacency_filter.drop_duplicates()
df_adjacency_filter = df_adjacency_filter.rename(columns = {'source_hex':'pickup_location_hex_8'})
df_adjacency_filter.head()

,pickup_location_hex_8,multicast_max_batch_size
10,883da1ba6dfffff,30
30,883da1034dfffff,30
31,883da1ad07fffff,5
46,883da1044dfffff,10
207,883da118d5fffff,30


In [76]:
df_accepted = df_accepted.merge(df_adjacency_filter, on = 'pickup_location_hex_8', how = 'left')
df_accepted.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,counter,multicast_max_batch_size
0,64919423e80c503f0a6d4726,NaN,883da112d9fffff,28.640282,77.082558,Delhi,883da112d1fffff,883da1176dfffff,28.610507,77.114565,28.637908,77.090801,172723,17,1715,Auto,17:00,2,20230620,onTheWay,None,1687262254108,accepted,6385d2484c014e83f4b30394,None,883da112d1fffff,1117.4,1.0,30
1,64919743fc04cd243acac84f,NaN,883da110c3fffff,28.577063,77.110474,Delhi,883da110ddfffff,883da115c3fffff,28.588634,77.253258,28.587704,77.118866,174043,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687263123431,accepted,600b92e97143235ff5c0b23a,None,883da110ddfffff,1379.8,3.0,30
2,6491955cc1b4474a4f66950c,NaN,883da11ab5fffff,28.443293,77.080841,Delhi,883da11ab5fffff,883da11abbfffff,28.453547,77.089450,28.442722,77.077400,173236,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687262561819,accepted,5fe2ddce026a4044e4ca9b91,None,883da11ab5fffff,458.7,1.0,10
3,64919875e205f4706879cc72,NaN,883da11ae3fffff,28.452248,77.050560,Delhi,883da11ae3fffff,883da11a83fffff,28.458678,77.071876,28.453086,77.044795,174549,17,1745,Auto,17:59,2,20230620,onTheWay,None,1687263355036,accepted,634a3b2093d5ea24f7438b0d,None,883da11ae3fffff,807.1,1.0,30
4,6491975ba123ae7aadac8d9a,NaN,883da1062bfffff,28.569397,77.327637,Delhi,883da1062bfffff,883da10629fffff,28.567341,77.321130,28.568413,77.327118,174107,17,1730,Auto,17:59,2,20230620,onTheWay,None,1687263075627,accepted,5dcae1680856f1461826a50a,None,883da1062bfffff,213.3,1.0,10


In [80]:
batch_wise_acceptance = df_accepted.groupby(['multicast_max_batch_size', 'counter']).size().reset_index()
batch_wise_acceptance = batch_wise_acceptance.rename(columns = {0:'accepted_pings'})
batch_wise_acceptance2 = batch_wise_acceptance.groupby(['multicast_max_batch_size'])['accepted_pings'].sum().reset_index()
batch_wise_acceptance2 = batch_wise_acceptance2.rename(columns = {'accepted_pings':'total_accepted_pings'})
batch_wise_acceptance = batch_wise_acceptance.merge(batch_wise_acceptance2, on = 'multicast_max_batch_size', how = 'left')
batch_wise_acceptance

,multicast_max_batch_size,counter,accepted_pings,total_accepted_pings
0,10,1.0,1604,2641
1,10,2.0,502,2641
2,10,3.0,206,2641
3,10,4.0,117,2641
4,10,5.0,74,2641
5,10,6.0,50,2641
6,10,7.0,30,2641
7,10,8.0,24,2641
8,10,9.0,12,2641
9,10,10.0,8,2641


In [81]:
batch_wise_acceptance.to_csv('delhi_batch_wise_acceptance.csv', index = False)


In [30]:
batch_wise_acceptance = pd.DataFrame(df_accepted['counter'].value_counts().reset_index())
batch_wise_acceptance.columns = ['batch_number', 'accepted_pings']
batch_wise_acceptance['total_accepted_pings'] = df_accepted.shape[0]
batch_wise_acceptance['%_of_accepted_pings'] = round(batch_wise_acceptance['accepted_pings']/batch_wise_acceptance['total_accepted_pings'], 4)
batch_wise_acceptance

,batch_number,accepted_pings,total_accepted_pings,%_of_accepted_pings
0,1.0,9429,15144,0.6226
1,2.0,2511,15144,0.1658
2,3.0,1353,15144,0.0893
3,4.0,731,15144,0.0483
4,5.0,510,15144,0.0337
5,6.0,283,15144,0.0187
6,7.0,137,15144,0.0090
7,8.0,78,15144,0.0052
8,9.0,33,15144,0.0022
9,10.0,20,15144,0.0013


In [31]:
batch_wise_acceptance.to_csv('delhi_batch_wise_acceptance.csv', index = False)

In [52]:
df_propagation['count'] = 1

In [39]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, drop_location_hex_8, pickup_location_hex_8, distance_final_distance, customer_id, customer_obj_mobile
from orders.order_logs_snapshot
where yyyymmdd in ('20230605', '20230606', '20230607')
and city_name in ('Hyderabad')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('8860a24b61fffff', '8860a24b6bfffff', '8860b196b1fffff', '8860a24b0dfffff', '8860b196b7fffff', '8860b196b3fffff', '8860a24b6dfffff',
 '8860a24b67fffff', '8860b19691fffff', '8860a24b65fffff', '8860a24b2dfffff', '8860b196bbfffff', '8860a24b05fffff', '8860a24b23fffff', '8860a24b69fffff',
   '8860a24b09fffff', '8860a24b29fffff', '8860b194dbfffff', '8860a24b63fffff', '8860a24b2bfffff', '8860b1969dfffff', '8860a24b21fffff', '8860a24b07fffff',
     '8860b19697fffff', '8860b19695fffff', '8860a24b01fffff', '8860a24b47fffff', '8860b194d9fffff', '8860a24b45fffff', '8860b19693fffff')
and hour in ('17', '18', '19', '20', '21')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [53]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,drop_location_hex_8,pickup_location_hex_8,distance_final_distance,customer_id,customer_obj_mobile
0,6480a65c9f628772885f1376,NaN,Hyderabad,8860a24b6bfffff,17.483747,78.388893,211636,21,2115,Auto,21:00,3,20230607,expired,None,0.0,"[""62975f9492702e0dd94c8914""]",8860a24b69fffff,8860a24b6bfffff,1.980,625ce256c3e605f6c4fe07be,8977907151
1,6480a661d48b012de40642e3,0.140,Hyderabad,8860a24b6bfffff,17.484764,78.388817,211641,21,2115,Auto,21:00,3,20230607,dropped,None,2.0,"[""6054c203ba88aa5e9064c50b"",""6241354de2879ad67...",8860a24b69fffff,8860a24b6bfffff,1.396,629c59020e8e58338fe25ff3,7702619435
2,6480a58d9f628772885f126b,0.631,Hyderabad,8860b196b1fffff,17.506567,78.415260,211309,21,2100,Auto,21:00,3,20230607,dropped,None,2.0,"[""63089b9e5f96473f0cea894b"",""5f044dcd86a506785...",8860b196bbfffff,8860b196b1fffff,2.844,645c7c854ce8f886a4406bbd,7386219530
3,6480a85f9f628772885f15f6,NaN,Hyderabad,8860a24b69fffff,17.488798,78.395355,212511,21,2115,Auto,21:00,3,20230607,expired,None,1.0,"[""611e046489020920a15353f0"",""5f0453b5819ef940d...",8860b196b7fffff,8860a24b69fffff,4.260,6249563d317867789665ec00,6281641351
4,6480a95e4c67352836f387c8,1.306,Hyderabad,8860a24b61fffff,17.498829,78.392441,212926,21,2115,Auto,21:00,3,20230607,dropped,None,3.0,"[""62a942927d79674e55e9118d"",""62a36c847d79674f8...",8860a24b69fffff,8860a24b61fffff,2.310,5f6c8c529c485a1e8e752762,8019030094


In [54]:
df_propagation_filter = df_propagation[df_propagation['data_orderId'].isin(df['order_id'].unique().tolist())]
df_propagation_filter.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter,count
5193812,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,63ea35ded6d9b21282e6a890,rider_acknowledged,17,1685964608623,1,1
4954696,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,61728683b9941b73252426c3,rider_acknowledged,17,1685964608774,1,1
5089962,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,6478ac31d189a0614fd7ffe7,rider_acknowledged,17,1685964609265,1,1
5109099,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,5f03356086a506c42a1bdf78,rider_acknowledged,17,1685964609662,1,1
3880010,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,60eeb4ced417d4225977e36a,rider_acknowledged,17,1685964609940,1,1


In [55]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [56]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [57]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [58]:
df_propagation_filter = df_propagation_filter.rename(columns = {'data_orderId':'order_id'})
df_propagation_filter = df_propagation_filter.merge(df_filter_level1[['order_id', 'final_order_status']], on = 'order_id', how = 'left')
df_propagation_filter.head()

,yyyymmdd,epoch,order_id,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter,count,final_order_status
0,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,63ea35ded6d9b21282e6a890,rider_acknowledged,17,1685964608623,1,1,dropped
1,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,61728683b9941b73252426c3,rider_acknowledged,17,1685964608774,1,1,dropped
2,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,6478ac31d189a0614fd7ffe7,rider_acknowledged,17,1685964609265,1,1,dropped
3,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,5f03356086a506c42a1bdf78,rider_acknowledged,17,1685964609662,1,1,dropped
4,20230605,1685964603440,647dc73b8ab6a81349c06ade,3c296b2e-b005-4726-a93c-ad3d0e9edbee,60eeb4ced417d4225977e36a,rider_acknowledged,17,1685964609940,1,1,dropped


In [59]:
total_pings = df_propagation_filter.groupby(['yyyymmdd', 'hh'])['count'].sum().reset_index()
net_orders = df_propagation_filter[df_propagation_filter['final_order_status'] == 'dropped'].groupby(['yyyymmdd', 'hh'])['order_id'].nunique().reset_index()
total_pings_dropped_orders = df_propagation_filter[df_propagation_filter['final_order_status'] == 'dropped'].groupby(['yyyymmdd', 'hh'])['count'].sum().reset_index()

total_pings = total_pings.rename(columns = {'count':'total_pings'})
total_pings_dropped_orders = total_pings_dropped_orders.rename(columns = {'count':'total_pings_dropped_orders'})
net_orders = net_orders.rename(columns = {'order_id':'net_orders'})

total_pings = total_pings.merge(total_pings_dropped_orders, on = ['yyyymmdd', 'hh'], how = 'left')
total_pings = total_pings.merge(net_orders, on = ['yyyymmdd', 'hh'], how = 'left')
total_pings.head()

,yyyymmdd,hh,total_pings,total_pings_dropped_orders,net_orders
0,20230605,17,11484,8630,481
1,20230605,18,16279,10092,554
2,20230605,19,22241,7838,446
3,20230605,20,19356,5848,340
4,20230605,21,20130,5272,300


In [60]:
total_pings['ME'] = round(total_pings['total_pings']/total_pings['net_orders'], 4)
total_pings['ME_dropped'] = round(total_pings['total_pings_dropped_orders']/total_pings['net_orders'], 4)
total_pings

,yyyymmdd,hh,total_pings,total_pings_dropped_orders,net_orders,ME,ME_dropped
0,20230605,17,11484,8630,481,23.8753,17.9418
1,20230605,18,16279,10092,554,29.3845,18.2166
2,20230605,19,22241,7838,446,49.8677,17.5740
3,20230605,20,19356,5848,340,56.9294,17.2000
4,20230605,21,20130,5272,300,67.1000,17.5733
5,20230606,17,10464,7951,468,22.3590,16.9893
6,20230606,18,13100,9260,516,25.3876,17.9457
7,20230606,19,17699,7859,423,41.8416,18.5792
8,20230606,20,15501,6435,338,45.8609,19.0385
9,20230606,21,14025,5170,273,51.3736,18.9377


In [22]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
1737,20230608,1686223801981,6481bbb93613ac7ea8a52f88,7fec62ea-55c4-4103-b414-b9095a72956f,63c64c27fff586a70e0cebd7,rider_acknowledged,17,1686223803932,1
11569,20230608,1686223801981,6481bbb93613ac7ea8a52f88,7fec62ea-55c4-4103-b414-b9095a72956f,647c1966cdea0ac9223cd3fd,rider_acknowledged,17,1686223804593,1
2319,20230608,1686223801981,6481bbb93613ac7ea8a52f88,7fec62ea-55c4-4103-b414-b9095a72956f,6222fa5c82ac75205c328206,rider_acknowledged,17,1686223804665,1
16,20230608,1686223801981,6481bbb93613ac7ea8a52f88,7fec62ea-55c4-4103-b414-b9095a72956f,604f94aa1630b428623b48ac,rider_acknowledged,17,1686223805976,1
5158,20230608,1686223801981,6481bbb93613ac7ea8a52f88,7fec62ea-55c4-4103-b414-b9095a72956f,62d4323b32a9212bde778ca5,rider_acknowledged,17,1686223805991,1


In [23]:
df_propagation_filter = df_propagation[['data_orderId', 'data_riderId', 'counter']]
df_propagation_filter.head()

,data_orderId,data_riderId,counter
1737,6481bbb93613ac7ea8a52f88,63c64c27fff586a70e0cebd7,1
11569,6481bbb93613ac7ea8a52f88,647c1966cdea0ac9223cd3fd,1
2319,6481bbb93613ac7ea8a52f88,6222fa5c82ac75205c328206,1
16,6481bbb93613ac7ea8a52f88,604f94aa1630b428623b48ac,1
5158,6481bbb93613ac7ea8a52f88,62d4323b32a9212bde778ca5,1


In [24]:
df_propagation_filter = df_propagation_filter.drop_duplicates()

In [25]:
#Merge this on df
df_propagation_filter = df_propagation_filter.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df_apr = df_apr.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8,estimated_accept_to_pickup_distance,counter
0,6481c03dc3889d46af8d9d26,NaN,883da11aa5fffff,28.434240,77.055037,Delhi,883da11aa1fffff,883da1112dfffff,28.489415,77.011870,28.438972,77.062790,171917,17,1715,Auto,17:00,4,20230608,new,None,1686224962209,rider_reject,63638610b6490e1f2f5b537d,None,883da11aa1fffff,1189.7,2
1,6481c299c3889d46af8da198,NaN,883da11aa5fffff,28.431310,77.057020,Delhi,883da11aa5fffff,883da1112dfffff,28.489415,77.011870,28.436775,77.054474,172921,17,1715,Auto,17:00,4,20230608,new,None,1686225569386,rider_reject,61ee6cd4ebef8294ac242d27,None,883da11aa5fffff,908.7,1
2,6481c85030f0ba3eb5a5eb8c,NaN,883da11aadfffff,28.439976,77.052472,Delhi,883da11aa1fffff,883da11ae1fffff,28.453949,77.038850,28.442194,77.056297,175344,17,1745,Auto,17:59,4,20230608,new,None,1686227032791,rider_reject,5fe95b1e4319a76867b78c6c,None,883da11aa1fffff,492.5,1
3,6481c71c1982f112e599a52c,NaN,883da118c9fffff,28.436443,77.072315,Delhi,883da11ab5fffff,883da118c9fffff,28.433812,77.073412,28.437632,77.081039,174836,17,1745,Auto,17:59,4,20230608,onTheWay,None,1686226720649,accepted,63256012b67a1624dc7cc68e,None,883da11ab5fffff,1099.2,2
4,6481c9b0bc90d94a27e09840,NaN,883da11abdfffff,28.448914,77.072386,Delhi,883da11abdfffff,883da10365fffff,28.451313,77.098116,28.450001,77.075325,175936,17,1745,Auto,17:59,4,20230608,onTheWay,None,1686227382179,accepted,62ad7ea4214ba0ca1cf2da50,None,883da11abdfffff,710.4,1


In [26]:
df_apr[df_apr['event_type'] == 'accepted']['counter'].value_counts()

counter
1    300
2     87
3     17
4     13
6      5
5      3
7      1
9      1
Name: count, dtype: int64

In [17]:
df_apr[df_apr['event_type'] == 'accepted'].shape

(358, 27)

In [27]:
300/(427)

0.702576112412178

In [28]:
167/236

0.7076271186440678

In [13]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, estimated_accept_to_pickup_distance
from orders.order_logs_snapshot
where yyyymmdd = '20230608'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('883da11a95fffff', '883da11abbfffff', '883da11a83fffff', '883da11ab5fffff', '883da11abdfffff', '883da11aa9fffff', '883da11aa1fffff',
 '883da11aa5fffff', '883da11ab9fffff', '883da11a87fffff', '883da118cbfffff', '883da11a81fffff', '883da11ab1fffff', '883da11a85fffff', '883da118c9fffff',
  '883da11ab7fffff', '883da11aabfffff', '883da11aa7fffff', '883da11aa3fffff', '883da118ddfffff')
and hour in ('17')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [14]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7


In [13]:
#Query data from order_logs_snapshot

q = """
select yyyymmdd, epoch, data_orderId, data_propagationBatchId, data_riderId, eventType, hh, updated_epoch from raw.kafka_dispatch_propagation_immutable
where data_serviceDetailId = '5c53562fceb6fc9241980547'
and yyyymmdd = '20230510'
and hh in ('17', '18', '19', '20', '21')
and eventType = 'rider_acknowledged'
"""


#Load data into pandas table
df_propagation_copy = pd.read_sql(q, connection)

In [14]:
df_propagation = df_propagation_copy.copy()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
0,20230510,1683728004312,645ba641ec315d1c8d2bd7ae,bb80701b-3d37-4f08-a785-149c947aadf3,636e036a16a8330067225cee,rider_acknowledged,19,1683728004312
1,20230510,1683728076711,645ba6b555ee7b32ef1b5170,2a264f49-c6ac-480e-b207-6db2959189a0,618205d216acba689a75f15e,rider_acknowledged,19,1683728076711
2,20230510,1683727635916,645ba4bd70e5ce05969cd4f7,23800730-303b-4e5d-b30f-16691aee6b49,60e6cf67569b260daa7398e7,rider_acknowledged,19,1683727635916
3,20230510,1683728006954,645ba659de453b02ef336618,74f9fc46-fcd2-45fb-9192-f8ce75ffdd50,60ed1dcf13ae1d7713b5443f,rider_acknowledged,19,1683728006954
4,20230510,1683727309457,645ba3bc6a343d115beac0a9,fb72744c-496f-4d84-b9c1-3619f62be1a6,635f6d09c6d7c2aa3e158a60,rider_acknowledged,19,1683727309457


In [15]:
df_propagation = df_propagation.sort_values(by = ['data_orderId', 'data_propagationBatchId', 'updated_epoch'])
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208


In [16]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914,1
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998,1
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114,1
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676,1
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208,1


In [17]:
df_propagation.head(20)

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914,1
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998,1
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114,1
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676,1
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208,1
391033,20230510,1683726917417,645b0c66908b9d588b5dc66f,fe0cf86e-ce13-4337-af6d-6afb5855c10e,642bffc70e17959cf78df534,rider_acknowledged,19,1683726917417,1
192147,20230510,1683722143183,645b2178af0d3209d269c3df,ebe83566-7f3a-4ebd-972a-bd9518290712,60f121df8bb15d81b65193f2,rider_acknowledged,18,1683722143183,1
549634,20230510,1683730536871,645b39b670e5ce05969bf500,8c3ec9be-8897-4e47-b353-6497fd241810,62ac9b667d79671ecaeba24e,rider_acknowledged,20,1683730536871,1
7989,20230510,1683730692786,645b39b670e5ce05969bf500,a7480d15-da57-4ff1-a354-605b2bc41723,6436b932303d7e7a54697cf4,rider_acknowledged,20,1683730692786,2
254495,20230510,1683730581854,645b39b670e5ce05969bf500,d17d70cd-14b1-497a-bab0-8211fb07640b,62ac9b667d79671ecaeba24e,rider_acknowledged,20,1683730581854,3


In [18]:
df_propagation[df_propagation['data_propagationBatchId'] == 'b8f09de1-5612-4e26-afdd-9b838e627092']

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,updated_epoch,counter
393665,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1683726307914,1
543576,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1683726307998,1
398398,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1683726309114,1
414195,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1683726309676,1
509157,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1683726310208,1


In [19]:
captains_pinged_per_batch = df_propagation.groupby(['data_propagationBatchId'])['data_riderId'].nunique().reset_index()
captains_pinged_per_batch = captains_pinged_per_batch.rename(columns = {'data_riderId':'captains_pinged'})
df_propagation_unique = df_propagation[['data_propagationBatchId', 'counter']]
df_propagation_unique = df_propagation_unique.drop_duplicates()
captains_pinged_per_batch = captains_pinged_per_batch.merge(df_propagation_unique, on = 'data_propagationBatchId', how = 'left')
captains_pinged_per_batch.head()

,data_propagationBatchId,captains_pinged,counter
0,00000fcb-368c-4802-9bcd-94bc5621dacf,1,1
1,00003e40-737e-4dbc-996a-4274bc48d782,2,1
2,00005ce3-c170-4138-ba7d-2505706c7a0e,2,1
3,000077c0-fd84-4fb4-b588-52489a01d9b8,10,1
4,000092fc-2f8b-492a-af0b-a0e76e7866e9,1,1


In [20]:
captains_pinged_per_batch.groupby(['counter'])['captains_pinged'].mean().reset_index()

,counter,captains_pinged
0,1,1.905736
1,2,1.782025
2,3,1.762390
3,4,1.801321
4,5,1.858209
5,6,1.688690
6,7,1.731382
7,8,1.753266
8,9,1.717557
9,10,1.775281


In [18]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,counter
400155,20230510,1683726307914,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60e99160e5cb35adea5900ff,rider_acknowledged,19,1
242489,20230510,1683726307998,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,5ebaca21350aa75c87bb7f34,rider_acknowledged,19,1
240032,20230510,1683726309114,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,632bf4134bdbff70462cb037,rider_acknowledged,19,1
229761,20230510,1683726309676,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,6319d9882badab5c7d3a3a5e,rider_acknowledged,19,1
482423,20230510,1683726310208,645a6ddc8b51b47a8f1d567f,b8f09de1-5612-4e26-afdd-9b838e627092,60295c2a19e4984b6c1e9771,rider_acknowledged,19,1


In [19]:
df_propagation_filter = df_propagation[['data_orderId', 'data_riderId', 'counter']]
df_propagation_filter.head()

,data_orderId,data_riderId,counter
400155,645a6ddc8b51b47a8f1d567f,60e99160e5cb35adea5900ff,1
242489,645a6ddc8b51b47a8f1d567f,5ebaca21350aa75c87bb7f34,1
240032,645a6ddc8b51b47a8f1d567f,632bf4134bdbff70462cb037,1
229761,645a6ddc8b51b47a8f1d567f,6319d9882badab5c7d3a3a5e,1
482423,645a6ddc8b51b47a8f1d567f,60295c2a19e4984b6c1e9771,1


In [20]:
#Merge this on df
df_propagation_filter = df_propagation_filter.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df = df.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,NaN
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0


In [21]:
df['counter'] = df['counter'].fillna(1)
df['counter_group'] = np.where(df['counter'] >= 5, '5 or more batches', df['counter'])
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter,counter_group
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0,2.0
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0,5 or more batches
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,1.0,1.0
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0,1.0
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0,1.0


In [22]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [23]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [24]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [32]:
order_status = df_filter_level1[df_filter_level1['counter_group'] == '2.0']
order_status = order_status[['order_id', 'final_order_status']]

In [43]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id, estimated_accept_to_pickup_distance, updated_epoch, event_type
from orders.order_logs_immutable
where yyyymmdd = '20230510'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('8861892433fffff', '886189251bfffff', '88618925c3fffff', '8861892589fffff', '886189242dfffff', '8861892421fffff', '88618925d3fffff',
 '88618925cdfffff', '88618925d1fffff', '886189255bfffff', '8861892553fffff', '88618925d5fffff', '88618925e3fffff', '88618925ebfffff', '88618924adfffff',
   '8861892513fffff', '88618924e7fffff', '88618925c5fffff', '88618925dbfffff', '88618925ddfffff', '88618925c7fffff', '88618924a5fffff', '88618924e5fffff',
     '88618925c9fffff', '8861892599fffff', '88618925d9fffff', '8861892437fffff', '88618925e9fffff', '88618925d7fffff', '88618925c1fffff', '8861892591fffff',
       '88618925cbfffff', '8861892581fffff', '8861892431fffff', '886189259dfffff', '886189258dfffff', '886189258bfffff', '886189259bfffff', '8861892435fffff',
         '8861892517fffff', '88618925e1fffff', '88618925edfffff', '88618925e7fffff')
and hour in ('17', '18', '19', '20', '21')
and event_type in ('accepted', 'rider_busy', 'rider_reject')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [44]:
df_copy = df_copy.merge(order_status, on = 'order_id', how = 'left')
df_copy.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status
0,645bc55913c08a4ec9300ca1,NaN,Bangalore,8861892437fffff,12.910453,77.612503,215457,21,2145,Auto,21:59,3,20230510,new,None,2.0,"[""629f19df1b57e4adbae9f447"",""61fe1f9c725c4cea8...",8861892437fffff,62cbb9b801d28934302b9682,1214.4,1683735926040,rider_busy,expiry_mapped
1,645bba478dbcb75a89aaf31b,NaN,Bangalore,8861892589fffff,12.933781,77.608307,210743,21,2100,Auto,21:00,3,20230510,new,None,0.0,"[""62d3f19e32a921678d77579e"",""642e67c41ecc3a9b6...",8861892589fffff,62d3f19e32a921678d77579e,437.9,1683733123970,rider_busy,NaN
2,645bc5db612b712367f9b7c6,NaN,Bangalore,88618925c7fffff,12.935354,77.614131,215707,21,2145,Auto,21:59,3,20230510,new,None,3.0,"[""60e94a1fac6ff47ed860ad5f"",""62d4cf8a24fad3312...",88618925c7fffff,6151938cf2602c3cb62cabc4,953.6,1683736130433,rider_reject,NaN
3,645bc5f3612b712367f9b7ee,NaN,Bangalore,88618925d7fffff,12.929103,77.591507,215731,21,2145,Auto,21:59,3,20230510,new,None,0.0,"[""5e5503e9dcc6a7fda2a2e67f"",""60e44ebf09aa996cb...",88618925d7fffff,60e44ebf09aa996cbb41180b,1090.9,1683736141453,rider_busy,NaN
4,645bb892888565720ae94a6f,NaN,Bangalore,8861892c19fffff,13.030947,77.648964,210026,21,2100,Auto,21:00,3,20230510,new,None,1.0,"[""62d793f9ceb77701571006bc"",""62b1760317af9a1ea...",88618925e7fffff,6295caae74c07130cc508775,589.7,1683732663471,rider_busy,expiry_mapped


In [45]:
df_copy = df_copy.dropna()
df_copy.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status
724,645bb146751ce46ce03ec71d,0.763,Bangalore,88618925e9fffff,12.939100,77.626884,202918,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,3.0,"[""62578fdd00b5f49e57ad65ea"",""5cf27fe9ca6e29211...",88618925e9fffff,63e3885d052c8a26c7edbc1e,776.4,1683731007711,rider_busy,COBRA
1110,645b8f5c3f9e605f9192f02d,1.925,Bangalore,8861892421fffff,12.909473,77.636635,180436,18,1800,Auto,18:00,3,20230510,new,Asked to buy items,7.0,"[""5de5d28eac56a62ec5e6185a"",""61d2e630de3a0254a...",886189242dfffff,60d2e01a05468956fbddd94a,1428.8,1683723161695,rider_busy,COBRA
1950,645b89a7b70c5170c58d26e5,0.757,Bangalore,88618925ebfffff,12.942251,77.621309,174015,17,1730,Auto,17:59,3,20230510,new,Unable to swipe Arrived,4.0,"[""5f1f96fdea5a8ba56679a603"",""623192433a5bed58b...",88618925ebfffff,63a5698d64e8005ff3fcd475,723.0,1683721181739,rider_reject,expiry_mapped
2963,645b963770e5ce05969cb64d,1.849,Bangalore,88618924a5fffff,12.917027,77.582852,183351,18,1830,Auto,18:59,3,20230510,new,Long Pickup,7.0,"[""61ef89c41772e473ff144182"",""60fd0164fe2f60ae4...",88618924a5fffff,6116409ef76f8bd4df482a2f,939.4,1683724042726,rider_busy,expiry_mapped
3054,645ba0a5de453b02ef335a6f,1.411,Bangalore,886189251bfffff,12.925212,77.637230,191821,19,1915,Auto,19:00,3,20230510,new,Long Pickup,6.0,"[""623c2999d26cb255e872d4ff"",""6449f5248fac8ad0b...",886189251bfffff,623c2999d26cb255e872d4ff,466.1,1683726895887,rider_busy,COBRA


In [46]:
df_copy2 = df_copy[df_copy['order_id'].isin(['645bc2a9ec315d1c8d2c0a7d', '645b995ec78d275898f7b767', '645baf766a343d115bead631'])]
df_copy2['timestamp'] = df_copy2['updated_epoch'].apply(lambda x: datetime.fromtimestamp(x/1000))
df_copy2 = df_copy2.sort_values(by = ['order_id', 'updated_epoch'])
df_copy2

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,updated_epoch,event_type,final_order_status,timestamp
34708,645b995ec78d275898f7b767,0.060,Bangalore,8861892553fffff,12.916032,77.651622,184718,18,1845,Auto,18:59,3,20230510,new,Drop location denied,2.0,"[""5e49f514b3394b0f2f2a3656"",""640185a7f22eada02...",8861892553fffff,616cf4654c6ba1f7ced8305a,681.4,1683724901048,rider_reject,dropped,2023-05-10 18:51:41.048
106595,645b995ec78d275898f7b767,0.060,Bangalore,8861892553fffff,12.916032,77.651622,184718,18,1845,Auto,18:59,3,20230510,onTheWay,Drop location denied,2.0,"[""5e49f514b3394b0f2f2a3656"",""640185a7f22eada02...",8861892553fffff,5e49f514b3394b0f2f2a3656,177.8,1683724905431,accepted,dropped,2023-05-10 18:51:45.431
98465,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,62a306a81284ca0edd09f4cf,968.1,1683730376738,rider_reject,COBRA,2023-05-10 20:22:56.738
97277,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,61cc08719ba37202862da8bc,1063.6,1683730380880,rider_reject,COBRA,2023-05-10 20:23:00.880
24471,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,6108eb0fb8bce11a8a1256b3,1913.8,1683730385758,rider_reject,COBRA,2023-05-10 20:23:05.758
32063,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,5be26f8f2b706c16e077f1cc,769.5,1683730386927,rider_reject,COBRA,2023-05-10 20:23:06.927
129617,645baf766a343d115bead631,1.843,Bangalore,88618925c3fffff,12.925103,77.612465,202134,20,2015,Auto,20:00,3,20230510,new,Heavy Traffic,7.0,"[""62ee41eab6ba02805f667cda"",""60fbb2d9fe2f607ff...",88618925c3fffff,60fbb2d9fe2f607ffe869f45,982.5,1683730394652,rider_reject,COBRA,2023-05-10 20:23:14.652
25280,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""60fa8812d417d4a5a...",88618925e1fffff,6149fa01de864c3f3a034fe1,332.0,1683735350385,rider_reject,expiry_mapped,2023-05-10 21:45:50.385
115031,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""60fa8812d417d4a5a...",88618925e1fffff,60fa8812d417d4a5a57dfa33,866.4,1683735378636,rider_reject,expiry_mapped,2023-05-10 21:46:18.636
88896,645bc2a9ec315d1c8d2c0a7d,0.742,Bangalore,88618925e1fffff,12.945374,77.626076,214329,21,2130,Auto,21:59,3,20230510,new,Long Pickup,3.0,"[""62aeeeb267cffab13cea2c33"",""610295906cd49257e...",88618925e1fffff,60e42d4209aa990f9b40fa77,1725.3,1683735422658,rider_busy,expiry_mapped,2023-05-10 21:47:02.658


In [47]:
df_copy2.to_csv('sample_orders.csv', index = False)

In [29]:
df_filter_level1

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,estimated_accept_to_pickup_distance,counter,counter_group,count,date,new_cancel_reason,expiry_type,final_order_status
0,645bae48bc677762c758e7c3,NaN,Bangalore,8861892591fffff,12.930825,77.580872,201632,20,2015,Auto,20:00,3,20230510,expired,None,7.0,"[""60ed2fb613ae1d748eb5528a"",""6013ae76edd74b702...",8861892591fffff,62a072577d796770ede1fe5f,1060.2,2.0,2.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
1,645bae4bde453b02ef33750e,NaN,Bangalore,8861892513fffff,12.931752,77.634494,201635,20,2015,Auto,20:00,3,20230510,expired,None,2.0,"[""60f40f138f3479e752195633"",""63b683460e10cd5a3...",8861892513fffff,63d405e015a98c9f1f0db9af,1882.5,6.0,5 or more batches,1,10,no_cancellation,expiry_mapped,expiry_mapped
2,645bae58888565720ae938e6,NaN,Bangalore,8861892433fffff,12.906545,77.606756,201648,20,2015,Auto,20:00,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""60685591a2f9c56b225129b7"",""62578fdd00b5f49e5...",8861892433fffff,,NaN,1.0,1.0,1,10,COBRA,no_expiry,COBRA
3,645baf8b888565720ae93b0d,0.438,Bangalore,88618925ddfffff,12.921208,77.604073,202155,20,2015,Auto,20:00,3,20230510,dropped,None,2.0,"[""62921b26444fa9ba4f782075"",""610a4a8a06f56f41f...",88618925ddfffff,631b7e788276172360bbb215,NaN,1.0,1.0,1,10,no_cancellation,no_expiry,dropped
4,645bb05bb70c5170c58d75f8,NaN,Bangalore,88618925c1fffff,12.923099,77.620018,202523,20,2015,Auto,20:00,3,20230510,expired,None,0.0,"[""6442088be2719e34f0671600"",""62578fdd00b5f49e5...",88618925c1fffff,6442088be2719e34f0671600,1122.7,1.0,1.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30305,645b97f4c78d275898f7b451,NaN,Bangalore,88618924e7fffff,12.904047,77.588232,184116,18,1830,Auto,18:59,3,20230510,customerCancelled,order cancelled before rider accepted,3.0,"[""5c7bba9f875c9853625f6f1b"",""60f165268f34791bc...",88618924e7fffff,5c7bba9f875c9853625f6f1b,533.2,1.0,1.0,1,10,COBRA,no_expiry,COBRA
30306,645b9a33c78d275898f7b928,NaN,Bangalore,88618925c9fffff,12.916768,77.623550,185051,18,1845,Auto,18:59,3,20230510,expired,None,2.0,"[""61067df8a31862ce6b59c3bf""]",88618925c9fffff,6222fe71c1a4e4e1e4645af4,1954.5,3.0,3.0,1,10,no_cancellation,expiry_mapped,expiry_mapped
30307,645b98276a343d115beaa7cc,NaN,Bangalore,88618925ebfffff,12.940269,77.620054,184207,18,1830,Auto,18:59,3,20230510,customerCancelled,order cancelled before rider accepted,2.0,"[""623192433a5bed58b8bec937"",""60efd298d417d4cd4...",88618925ebfffff,643106272548e37cc9e53fa1,1295.4,2.0,2.0,1,10,COBRA,no_expiry,COBRA
30308,645b9830ec315d1c8d2bba01,NaN,Bangalore,88618925d9fffff,12.914090,77.599678,184216,18,1830,Auto,18:59,3,20230510,expired,None,2.0,"[""615163be90fbcb7e26d0ac55"",""60fd0164fe2f60ae4...",88618925d1fffff,6116409ef76f8bd4df482a2f,1237.6,4.0,4.0,1,10,no_cancellation,expiry_mapped,expiry_mapped


In [25]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'counter_group'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'counter_group', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230510,1.0,0.0252,0.5712,0.002,0.0996,0.3015,0.0002,9967
1,20230510,2.0,0.0412,0.5027,0.000,0.1310,0.3242,0.0000,4732
2,20230510,3.0,0.0361,0.3607,0.000,0.0835,0.5193,0.0000,5099
3,20230510,4.0,0.0218,0.1646,0.000,0.0467,0.7665,0.0000,5584
4,20230510,5 or more batches,0.0163,0.0737,0.000,0.0336,0.8760,0.0000,4911


In [66]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df

,yyyymmdd,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,20230417,0.0504,0.3347,0.0000,0.1658,0.4485,0.0,32180
1,20230418,0.0500,0.3283,0.0001,0.1693,0.4516,0.0,32213


In [64]:
#Get the average ETA and FM
df_dropped = df_filter_level1[df_filter_level1['final_order_status'] == 'dropped']
fm = df_dropped.groupby(['yyyymmdd', 'counter_group'])['accept_to_pickup_distance'].median().reset_index()
fm = fm.rename(columns = {'accept_to_pickup_distance':'FM'})
eta = df_dropped.groupby(['yyyymmdd', 'counter_group'])['eta'].median().reset_index()
metrics_df = metrics_df.merge(fm, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df = metrics_df.merge(eta, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df.head()

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0


In [65]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0
5,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,0.6870,3.0
6,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,0.7970,3.0
7,20230418,3.0,0.0625,0.3872,0.0000,0.1792,0.3707,0.0000,4783,0.7330,3.0
8,20230418,4.0,0.0376,0.2154,0.0000,0.0821,0.6644,0.0000,5348,0.7980,3.0
9,20230418,5 or more batches,0.0223,0.0902,0.0000,0.0465,0.8409,0.0000,7362,0.7890,3.0


In [57]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM_x,eta_x,FM_y,eta_y
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,3.404303,3.072228,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,16.989028,3.336893,0.8165,3.0
2,20230417,3 or more batches,0.0377,0.2133,0.0000,0.0891,0.6595,0.0000,17226,3.227829,3.340065,0.7570,3.0
3,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,3.313941,3.058516,0.6870,3.0
4,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,16.525852,3.299471,0.7970,3.0
5,20230418,3 or more batches,0.0380,0.2097,0.0000,0.0936,0.6584,0.0000,17493,7.587716,3.266178,0.7560,3.0
